# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto baquerizo moreno,-0.9000,-89.6000,23.75,92,58,4.16,EC,1713783374
1,1,bredasdorp,-34.5322,20.0403,23.82,50,5,4.25,ZA,1713783484
2,2,savanna-la-mar,18.2190,-78.1332,24.97,87,3,1.73,JM,1713783487
3,3,tazacorte,28.6290,-17.9293,21.54,76,75,3.09,ES,1713783489
4,4,arraial do cabo,-22.9661,-42.0278,23.92,69,0,3.60,BR,1713783492


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,    
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map
map_plot_1


C:\Users\erink\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Define temperature, humidity and cloudiness
filtered_data_df = city_data_df.loc[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 30) & (city_data_df['Humidity'] >= 50) & (city_data_df['Cloudiness'] == 0)]
row_count = filtered_data_df.shape[0]
print(row_count)

# Drop any rows with null values
filtered_data_df = filtered_data_df.dropna()
# Display sample data
filtered_data_df.head()

# count rows
row_count = filtered_data_df.shape[0]
print(row_count)


14
14


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']  # Specify the columns you want to keep
hotel_df = filtered_data_df[columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
4,arraial do cabo,BR,-22.9661,-42.0278,69,
44,campos dos goytacazes,BR,-21.7500,-41.3000,94,
59,moreira sales,BR,-24.0622,-53.0069,82,
126,gwadar,PK,25.1216,62.3254,62,
152,morondava,MG,-20.2833,44.2833,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "api_key":geoapify_key,
    "limit":14
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
arraial do cabo - nearest hotel: No hotel found
campos dos goytacazes - nearest hotel: Hotel Terrazzo
moreira sales - nearest hotel: No hotel found
gwadar - nearest hotel: Sadaf Resorts
morondava - nearest hotel: Hotel Sharon Menabe
san luis de la loma - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
colares - nearest hotel: Hotel Arribas
ribas do rio pardo - nearest hotel: Hotel São Francisco
ilhabela - nearest hotel: Vila Kebaya
bataipora - nearest hotel: Hotel Estoril
kruisfontein - nearest hotel: No hotel found
embu - nearest hotel: Almenat Embu das Artes Sao Paulo, Tapestry Collection by Hilton
mangrol - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
4,arraial do cabo,BR,-22.9661,-42.0278,69,No hotel found
44,campos dos goytacazes,BR,-21.7500,-41.3000,94,Hotel Terrazzo
59,moreira sales,BR,-24.0622,-53.0069,82,No hotel found
126,gwadar,PK,25.1216,62.3254,62,Sadaf Resorts
152,morondava,MG,-20.2833,44.2833,60,Hotel Sharon Menabe


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,    
    size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)